In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
directors_df = pd.read_csv('../data/train/directors.tsv', sep='\t') # 감독 정보
genres_df = pd.read_csv('../data/train/genres.tsv', sep='\t') # 장르 정보
titles_df = pd.read_csv('../data/train/titles.tsv', sep='\t') # 영화 제목 정보
writers_df = pd.read_csv('../data/train/writers.tsv', sep='\t') # 각본가 정보
years_df = pd.read_csv('../data/train/years.tsv', sep='\t') # 개봉 연도 정보

train_df = pd.read_csv('../data/train/train_ratings.csv') # 전체 학습 데이터

In [ ]:
# 기본 정보 확인
print(train_df.info())
print(train_df.head())

# 결측치 확인
print(train_df.isnull().sum())

# 중복 행 확인
print(train_df[train_df.duplicated()])

In [ ]:
# (user, item) 쌍 중복 확인
duplicated_rows = train_df[train_df.duplicated(subset=['user', 'item'])]
print(duplicated_rows)


한 유저가 같은 영화를 두 번 이상 본 기록은 없다.
-> 예측 시 본 영화 제외할 지 안 할 지

In [ ]:
# time 컬럼의 최소값과 최대값
min_time = train_df['time'].min()
max_time = train_df['time'].max()

print(f"Minimum time: {min_time}")
print(f"Maximum time: {max_time}")

In [ ]:
# 고유 유저 수
unique_users = train_df['user'].nunique()
print(f"Unique users: {unique_users}")

# 영화별 시청한 유저 수
items_counts = train_df['item'].value_counts() # 같은 사람이 여러 번 시청한 경우도 포함

# 상위 10개 영화 정보 추출
top_10_items = items_counts.head(10)
top_10_items_df = top_10_items.reset_index()  # Series를 DataFrame으로 변환
top_10_items_df.columns = ['item', 'user_count']  # 컬럼 이름 설정

# titles_df와 병합하여 제목 추가
top_10_items_with_titles = top_10_items_df.merge(titles_df, on='item', how='left')

# 결과 출력
print(top_10_items_with_titles[['item', 'title', 'user_count']])


In [ ]:
user_movie_count = train_df['user'].value_counts()
print(user_movie_count.describe())  # 사용자별 영화 시청 수의 통계
print(user_movie_count.head(10))   # 상위 10명의 사용자

# 시각화
# Binwidth 설정
binwidth = 10  # 각 bin의 크기
bins = np.arange(0, user_movie_count.max() + binwidth, binwidth)  # 0부터 최대값까지 bin 생성

# 히스토그램 생성
plt.figure(figsize=(10, 6))
user_movie_count.plot(kind='hist', bins=bins, color='skyblue')

# 그래프 설정
plt.title('Histogram with Binwidth = 10', fontsize=14)
plt.xlabel('Number of Movies Watched', fontsize=12)
plt.ylabel('Number of Users', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


가장 적게 본 유저가 16개

In [ ]:
# x <= 500 데이터 필터링
filtered_user_movie_count = user_movie_count[user_movie_count <= 500]

# Binwidth 설정
binwidth = 1  # 각 bin의 크기
bins = np.arange(0, filtered_user_movie_count.max() + binwidth, binwidth)  # 0부터 최대값까지 bin 생성

# 히스토그램 생성
plt.figure(figsize=(10, 6))
filtered_user_movie_count.plot(kind='hist', bins=bins, color='skyblue')

# 그래프 설정
plt.title(f'Histogram with Binwidth = {binwidth} (x <= 500)', fontsize=14)
plt.xlabel('Number of Movies Watched', fontsize=12)
plt.ylabel('Number of Users', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# datetime 칼럼 추가
train_df['datetime'] = pd.to_datetime(train_df['time'], unit='s')

# datetime에서 월 단위로 추출한 별도의 칼럼 추가
train_df['month'] = train_df['datetime'].dt.to_period('M')  # 월 단위 추가

# 월 단위 시청 기록 수
monthly_distribution = train_df['month'].value_counts().sort_index()

# 시각화
plt.figure(figsize=(12, 6))
monthly_distribution.plot(kind='line', color='coral')
plt.title('Monthly Movie Watch Time Distribution', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Watches', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# 시청 기록 수 기준으로 내림차순 정렬
monthly_distribution_sorted = monthly_distribution.sort_values(ascending=False)

# 결과 출력
print(monthly_distribution_sorted)

In [ ]:
# 2008-10 데이터를 필터링
filtered_items = train_df[train_df['month'] == '2008-10']['item']

# 아이템별 시청 횟수 계산
item_counts = filtered_items.value_counts().head(15)

# 시청 횟수 상위 아이템을 DataFrame으로 변환
item_counts_df = item_counts.reset_index()
item_counts_df.columns = ['item', 'view_count']

# 영화 제목 정보와 병합
result = item_counts_df.merge(titles_df, on='item', how='left')

# 결과 출력
print(result[['item', 'title', 'view_count']])


개봉한지 오래된 영화들의 시청 수도 상당히 높다.

In [ ]:
import matplotlib.pyplot as plt

# datetime 칼럼 추가
train_df['datetime'] = pd.to_datetime(train_df['time'], unit='s')

# datetime에서 월 단위로 추출한 별도의 칼럼 추가
train_df['month'] = train_df['datetime'].dt.to_period('M')  # 월 단위 추가

# 월 단위 시청 기록 수
monthly_distribution = train_df['month'].value_counts().sort_index()

# 특정 item('2571')의 월별 시청 기록 추이 계산
item_2571_distribution = (
    train_df[train_df['item'] == 2571]
    .groupby('month')
    .size()
    .sort_index()
)

# 시각화: 두 개의 y축 설정
fig, ax1 = plt.subplots(figsize=(12, 6))

# 첫 번째 y축 (전체 영화)
ax1.plot(monthly_distribution.index.astype(str), monthly_distribution, color='coral', label='All Movies')
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('All Movies Watches', fontsize=12, color='coral')
ax1.tick_params(axis='y', labelcolor='coral')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# x축 눈금 위치와 회전 조정
ax1.set_xticks(range(len(monthly_distribution.index)))
ax1.set_xticklabels(monthly_distribution.index.astype(str), rotation=90, ha='right')

# 두 번째 y축 (item 2571)
ax2 = ax1.twinx()
ax2.plot(item_2571_distribution.index.astype(str), item_2571_distribution, color='blue', label='Item 2571')
ax2.set_ylabel('Item 2571 Watches', fontsize=12, color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

# 제목 및 레이아웃 설정
fig.suptitle('Monthly Movie Watch Time Distribution', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# datetime 칼럼 추가
train_df['datetime'] = pd.to_datetime(train_df['time'], unit='s')

# datetime에서 월 단위로 추출한 별도의 칼럼 추가
train_df['month'] = train_df['datetime'].dt.to_period('M')  # 월 단위 추가

# 월 단위 시청 기록 수
monthly_distribution = train_df['month'].value_counts().sort_index()

# 비교할 아이템 목록
items_to_plot = [2571, 2959, 296, 318, 356]
colors = ['blue', 'green', 'red', 'orange', 'purple']

# 특정 아이템의 월별 시청 기록 추이 계산
item_distributions = {
    item: train_df[train_df['item'] == item].groupby('month').size().sort_index()
    for item in items_to_plot
}

# 영화 제목 가져오기
titles = titles_df.set_index('item').loc[items_to_plot, 'title']

# 시각화: 두 개의 y축 설정
fig, ax1 = plt.subplots(figsize=(12, 6))

# 첫 번째 y축 (전체 영화)
ax1.plot(monthly_distribution.index.astype(str), monthly_distribution, color='coral', label='All Movies')
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('All Movies Watches', fontsize=12, color='coral')
ax1.tick_params(axis='y', labelcolor='coral')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# x축 눈금 회전
ax1.set_xticks(range(len(monthly_distribution.index)))
ax1.set_xticklabels(monthly_distribution.index.astype(str), rotation=90, ha='right')

# 두 번째 y축 설정
ax2 = ax1.twinx()

# 각 아이템의 그래프 추가
for item, color in zip(items_to_plot, colors):
    ax2.plot(item_distributions[item].index.astype(str), item_distributions[item], label=titles[item], color=color)

# y축 제거
ax2.tick_params(axis='y', labelright=False)

# 영화 이름을 그래프와 같은 색으로 표시
for item, color in zip(items_to_plot, colors):
    last_month = item_distributions[item].index[-1].strftime('%Y-%m')
    last_value = item_distributions[item].iloc[-1]
    ax2.text(len(monthly_distribution) - 1, last_value, titles[item], fontsize=10, color=color, ha='left')

# 제목 및 레이아웃 설정
fig.suptitle('Monthly Movie Watch Time Distribution', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
user_11_movies = train_df[train_df['user'] == 11].sort_values(by='time')
print(user_11_movies)

# 시각화: 특정 사용자의 시청 기록
plt.figure(figsize=(12, 6))
plt.plot(user_11_movies['datetime'], user_11_movies['item'], marker='o', linestyle='-', color='blue')
plt.title('User 11 Watch History', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Movie ID', fontsize=12)
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


시청한 영화가 한 번에 업데이트된다고 볼 수 있다. -> 시계열 정보 신뢰 가능?